In [5]:
import sys, os
import numpy as np

from time import sleep

## Point to the backend function scripts
sys.path.insert(1, "../DeviceControl")

from NEXUSFunctions import * #control NEXUS fridge

In [7]:
## Initialize the NEXUS temperature servers
nf1 = NEXUSTemps(server_ip="192.168.0.31",server_port=11031) ## free, COLD PLATE, STILL
nf2 = NEXUSTemps(server_ip="192.168.0.32",server_port=11032) ## MIXING CHAMBER, 2ND STAGE, 1ST STAGE
nf3 = NEXUSTemps(server_ip="192.168.0.34",server_port=11034) ## Heater on MC 

In [8]:
## Temperature scan settings [K]
Temp_base =  11e-3
Temp_min  =  20e-3
Temp_max  = 350e-3
Temp_step =  10e-3

## Temperature stabilization params
tempTolerance =   1e-4     ## K
sleepTime     =  10        ## sec
stableTime    = 180        ## sec

Temps = np.arange(Temp_min,Temp_max+Temp_step,Temp_step)
Temps = Temps[::-1]

if (Temp_base) < Temps[-1]:
    Temps = np.append(Temps,Temp_base)
    
print(Temps*1e3,"mK",len(Temps))

[350. 340. 330. 320. 310. 300. 290. 280. 270. 260. 250. 240. 230. 220.
 210. 200. 190. 180. 170. 160. 150. 140. 130. 120. 110. 100.  90.  80.
  70.  60.  50.  40.  30.  20.  11.] mK 35


In [9]:
def temp_change_and_wait(new_sp):

    print("CHANGING SETPOINT TO",new_sp,"K")
    nf3.setSP(new_sp)

    cTemp=float(nf3.getTemp())
    print("Waiting for Fridge to Reach Temperature")
    print("Monitoring temp every",sleepTime,"seconds")
    print("...",cTemp*1e3,"mK")
    terr = new_sp-cTemp
    while(np.abs(terr) > tempTolerance):
        sleep(sleepTime)
        try:
            cTemp=float(nf3.getTemp())
            terr = new_sp-cTemp
            print("...",cTemp*1e3,"("+terr*1e3+")","mK")
        except:
            print("Socket Failed, skipping reading")

    print("Holding at current temp for",stableTime,"seconds")
    sleep(stableTime)
    print("Done.")

    return 0

In [12]:
temp_change_and_wait(0.350)

CHANGING SETPOINT TO 0.35 K
Changing setpoint to 0.35
Waiting for Fridge to Reach Temperature
Monitoring temp every 10 seconds
... 0.35 mK
Holding at current temp for 180 seconds


0

In [ ]:
np.array([nf1.getResistance(), nf2.getResistance()])

In [ ]:
np.array([nf1.getTemp(), nf2.getTemp()])